**Packages**

In [1]:
import re
import numpy as np

import spacy

In [2]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [3]:
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.corpus import stopwords

In [4]:
import imp

tools = imp.load_source('tools', '../lda_tools.py')

In [5]:
import tqdm

**The data**

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('../data/prisoners_dilemma_articles_meta_data_clean.csv')

In [8]:
data = df[['abstract', 'unique_key', 'title', 'date']]

In [9]:
data = data.drop_duplicates()

In [10]:
data = data.reset_index(drop=True)

We want to perform the analysis yearly. We will write a function to perform the analysis but also clean the data
and create the corpus each time.

In [14]:
stop_words = stopwords.words('english')
mallet_path = '/Users/storm/rsc/mallet-2.0.8/bin/mallet'

In [20]:
def get_model_and_opt_num_of_topics(yearly_dataset, limit, step=1, start=2):
    
    # clean
    words = list(tools.sentences_to_words(yearly_data['abstract'].values))
#     bigram = gensim.models.Phrases(words, min_count=5, threshold=100)
#     bigram_mod = gensim.models.phrases.Phraser(bigram)
    lemmatized_words = tools.clean_words(words, stop_words)
    
    # corpus
    id2word = corpora.Dictionary(lemmatized_words)
    texts = lemmatized_words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    model_list, coherence_values = tools.compute_coherence_values(limit=limit,
                                                                mallet_path=mallet_path,
                                                                dictionary=id2word,
                                                                corpus=corpus,
                                                                texts=lemmatized_words,
                                                                step=step, start=start)
    
    max_coherence_value = max(coherence_values)
    max_index = coherence_values.index(max_coherence_value)
    
#     num_of_topics = list(range(start, limit, step))

    return model_list, max_index, coherence_values

In [21]:
years = sorted(data.date.unique())

In [22]:
len(years)

58

In [ ]:
results = []
for year in tqdm.tqdm_notebook(years[:5]):
    yearly_data = data[data['date'] <= year]
    
    result = get_model_and_opt_num_of_topics(yearly_data, limit=15, start=3)
    
    results.append(result)

//anaconda3/envs/literature/lib/python3.6/site-packages/tqdm/__init__.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  TqdmDeprecationWarning)


//anaconda3/envs/literature/lib/python3.6/site-packages/gensim/models/wrappers/ldamallet.py:379: RuntimeWarning: invalid value encountered in true_divide
  return topics / topics.sum(axis=1)[:, None]


In [ ]:
len(yearly_data['title'].unique())

In [ ]:
len(df['title'].unique())

In [ ]:
num_of_topics = list(range(2, 7, 1))

In [ ]:
models, max_indices, coh_val = zip(*results)

In [ ]:
coh_val[1]

In [ ]:
plt.plot([num_of_topics[index] for index in max_indices], '-o');